# 线程

Python中线程会在一个单独的系统级别线程中执行（比如一个POSIX线程或者一个Windows线程）.这些线程将由操作系统来全权管理。线程一旦启动，将独立执行直到目标函数返回。

### 常用用法

* t.is_alive() : 查询一个线程对象的状态，看它是否还在执行.
* t.join(): 把一个线程加入到当前线程，并等待它终止. Python 解释器在所有线程都终止后才继续执行代码剩余的部分.
* daemon: 对于需要长时间运行的线程或者需要一直运行的后台任务，可以用后台线程（也称为守护线程）,后台线程无法等待，这些线程会在主线程终止时自动销毁

```python
t = Thread(target = func, args(1,), daemon = True)
t.start()
```    
    

#### 小结：
* 后台线程无法等待，不过，后台线程会在主线程终止时自动销毁。
* 你无法结束一个线程，无法给它发送信号，无法调整它的调度，也无法执行其他高级操作。如果需要这些特性，需要自己添加。比如说，你需要终止线程，那么这个线程必须通过编程在某个特定点轮询来退出
* 如果线程执行一些像 I/O 这样的阻塞操作，那么通过轮询来终止线程将使得线程之间的协调变得非常棘手。比如，如果一个线程一直阻塞在一个 I/O 操作上，它就永远无法返回，也就无法检查自己是否已经被结束了。要正确处理这些问题，需要利用超时循环来小心操作线程。

使用threading模块创建多线程有两种方式:
* 传入一个函数并创建一个thread实例,然后调用start方法运行
* 继承threading.Thread类,重写__init__和run方法

In [1]:
# 法一
import random
import time
import threading

def thread_run(urls):
    print("current {} is runnging...".format(threading.current_thread().name))
    for url in urls:
        print("{} ----->>> {}".format(threading.current_thread().name, url))
        time.sleep(random.random())
    print("{} ended.".format(threading.current_thread().name))

print("{} is running...".format(threading.current_thread().name))
t1 = threading.Thread(target=thread_run, name='thread_1', args=(['url1', 'url2', 'url3'], ))
t2 = threading.Thread(target=thread_run, name='thread_2', args=(['url4', 'url5', 'url6'], ))

t1.start()
t2.start()

t1.join()
t2.join()

print("{} is ended.".format(threading.current_thread().name))

MainThread is running...
current thread_1 is runnging...
thread_1 ----->>> url1
current thread_2 is runnging...
thread_2 ----->>> url4
thread_1 ----->>> url2
thread_2 ----->>> url5
thread_2 ----->>> url6
thread_1 ----->>> url3
thread_2 ended.
thread_1 ended.
MainThread is ended.


In [2]:
import random
import time
import threading

class mythread(threading.Thread):
    def __init__(self, name, urls):
        # super(mythread, self).__init__(name=name)
        # 这里显式调用父类的初始化方法,区别于super()方法,虽然效果一样
        threading.Thread.__init__(self, name=name)
        self.urls = urls

    def run(self):
        print("current {} is runnging...".format(threading.current_thread().name))
        for url in self.urls:
            print("{} ----->>> {}".format(threading.current_thread().name, url))
            time.sleep(random.random())
        print("{} ended.".format(threading.current_thread().name))

print("{} is running...".format(threading.current_thread().name))
t1 = mythread(name='thread_1', urls=['url1', 'url2', 'url3'])
t2 = mythread(name='thread_2', urls=['url4', 'url5', 'url6'])

t1.start()
t2.start()

t1.join()
t2.join()

print("{} is ended.".format(threading.current_thread().name))

MainThread is running...
current thread_1 is runnging...
thread_1 ----->>> url1
current thread_2 is runnging...
thread_2 ----->>> url4
thread_1 ----->>> url2
thread_2 ----->>> url5
thread_1 ----->>> url3
thread_2 ----->>> url6
thread_1 ended.
thread_2 ended.
MainThread is ended.


### 线程间通信

一个线程向另外一个线程发送数据最安全的方式应该就是queue库中的队列

先看一下使用例子，这里是一个简单的生产者和消费者模型：

In [6]:
from queue import Queue
from threading import Thread
import random
import time


_sentinel = object()


def producer(out_q):
    n = 5
    while n:
        time.sleep(1)
        data = random.randint(0, 10)
        out_q.put(data)
        print("生产者生产了数据{0}\n".format(data))
        n -= 1
    out_q.put(_sentinel)


def consumer(in_q):
    while True:
        data = in_q.get()
        print("消费者消费了{0}".format(data))
        if data is _sentinel:
#             in_q.put(_sentinel)
            break


q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))

t1.start()
t2.start()

生产者生产了数据2
消费者消费了2

生产者生产了数据4
消费者消费了4

生产者生产了数据4
消费者消费了4

生产者生产了数据0
消费者消费了0

生产者生产了数据5
消费者消费了5

消费者消费了<object object at 0x000001F8278EBC60>


上述代码中设置了一个特殊值_sentinel用于当获取到这个值的时候终止执行

关于queue的功能有个需要注意的地方：
* Queue对象虽然已经包含了必要的锁，主要有q.put和q.get
* q.size(),q.full(),q.empty()等方法不是线程安全的
* 使用队列进行线程通信是一个单向、不确定的过程。通常情况下，是没有办法知道接收数据的线程是什么时候接收到的数据并开始工作的。但是队列提供了一些基本的特性：q.task_done()和q.join()
* 如果一个线程需要在另外一个线程处理完特定的数据任务后立即得到通知，可以把要发送的数据和一个Event放到一起使用

### 线程中的event

线程有一个非常关键的特性：每个线程都是独立运行的，且状态不可预测

如果程序中的其他线程需要通过判断每个线程的状态来确定自己下一步的操作，这时线程同步问题就会比较麻烦。

解决方法：使用threading库中的Event

Event对象包含一个可由线程设置的信号标志，它允许线程等待某些事件的发生。在初始化状态下，event对象中的信号标志被设置为假。
如果有线程等待一个event对象，而这个event的标志为假，这个线程将一直被阻塞知道该标志为真。一个线程如果把event对象的标志设置为真，就会唤醒所有等待这个event对象的线程。

一个例子：

In [7]:
from threading import Thread, Event
import time


def countdown(n, started_evt):
    print("countdown starting")
    # set将event的标识设置为True
    started_evt.set()
    while n > 0:
        print("T-mins", n)
        n -= 1
        time.sleep(1)

# 初始化的started_evt为False
started_evt = Event()
print("Launching countdown")
t = Thread(target=countdown, args=(10, started_evt,))
t.start()
# 会一直等待直到event的标志为True的时候
started_evt.wait()
print("countdown is running")

Launching countdown
countdown starting
T-mins 10
countdown is running
T-mins 9
T-mins 8
T-mins 7
T-mins 6
T-mins 5
T-mins 4
T-mins 3
T-mins 2
T-mins 1


实际用event对象最好是单次使用，创建一个event对象，让某个线程等待这个对象，一旦对象被设置为True，就应该丢弃它，我们虽然可以通过clear()方法重置event对象，但是这个没法确保安全的清理event对象并对它进行重新的赋值。会发生错过事件，死锁等各种问题。

event对象的一个重要特点:它被设置为True时会唤醒**所有等待它的线程**

如果唤醒**单个线程**的最好用Condition或信号量Semaphore

### 线程中的condition

condition变量总是和与某种锁相关联;可以传入，或默认创建一个。当多个condition变量共享同一个锁时，传入一个很有用。锁是condition变量的一部分：你不必单独跟踪它。

必须在获得关联锁的情况下调用其他方法。 wait（）方法释放锁，然后阻塞，直到另一个线程通过调用notify（）或notify_all（）唤醒它。一旦被唤醒，wait（）重新获得锁并返回。也可以指定超时。

注意: notify() and notify_all()这两个方法，不会释放锁，这意味着线程或者被唤醒的线程不会立刻执行wait()

通过Conditon对象实现一个周期定时器的功能，每当定时器超时的时候，其他线程都可以检测到，代码例子如下：

In [14]:
import threading
import time


class PeriodicTimer(object):
    """这里做了一个定时器"""

    def __init__(self, interval):
        self._interval = interval
        print('设置定时间隔 {} s'.format(self._interval))
        self._flag = 0
        self._cv = threading.Condition()

    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True
        t.start()

    def run(self):
        while True:
            time.sleep(self._interval)
            with self._cv:
                # 这个点还是非常有意思的^=
                self._flag ^= 1
                self._cv.notify_all()

    def wait_for_tick(self):
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()


# 下面两个分别为两个需要定时执行的任务
def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus {}'.format(nticks))
        nticks -= 1


def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting up {}'.format(n))
        n += 1


ptimer = PeriodicTimer(3)
ptimer.start()

threading.Thread(target=countdown, args=(10,)).start()
threading.Thread(target=countup, args=(5,)).start()

设置定时间隔 3 s
T-minus 10Counting up 0

Counting up 1T-minus 9

T-minus 8Counting up 2

T-minus 7Counting up 3

T-minus 6Counting up 4

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


### 线程锁

要在多线程中安全使用可变对象，需要使用threading库中的Lock对象

In [17]:
import threading


class SharedCounter(object):

    def __init__(self, initial_value=0):
        self._value = initial_value
        self._value_lock = threading.Lock()


    def incr(self, delta = 1):
        with self._value_lock:
            self._value += delta
            print('incr', self._value)

    def decr(self, delta=1):
        with self._value_lock:
            self._value -= delta
            print('decr', self._value)
            
sc = SharedCounter()

for i in range(5):
    sc.incr()
    if sc._value > 2:
        sc.decr()

incr 1
incr 2
incr 3
decr 2
incr 3
decr 2
incr 3
decr 2


这个例子中的锁是一个类变量，也就是所有实例共享的类级锁，这样就保证了一次只有一个线程可以调用这个类的方法。与标准锁不同的是已经持有这个锁的方法再调用同样适用这个锁的方法时，无需再次获取锁，例如上面例子中的decr方法。

优点：无论这个类有多少实例都使用一个锁。因此在需要使用大量使用计数器的情况下内存效率更高。
缺点：在程序中使用大量线程并频繁更新计数器时会有竞争用锁的问题。

### 信号量

信号量对象是一个建立在共享计数器基础上的同步原语

如果计数器不为0,with语句将计数器减1，线程被允许执行。with语句执行结束后，计数器加1。

如果计数器为0，线程将被阻塞，直到其他线程结束并将计数器加1。

但是信号量不推荐使用，增加了复杂性，影响程序性能。

所以信号量更适用于那些需要在线程之间引入信号或者限制的程序。例如限制一段代码的并发量.

In [ ]:
from threading import Semaphore
import requests


_fetch_url_sema = Semaphore(5)


def fetch_url(url):
    with _fetch_url_sema:
        return requests.get(url)

### 死锁问题

在多线程程序中，死锁问题很大一部分是由于**多线程同时获取多个锁**造成的。
举个例子：一个线程获取一个第一个锁，在获取第二个锁的时候发生阻塞，那么这个线程就可能阻塞其他线程执行，从而导致整个程序假死。

一种解决方法：为程序中每一个锁分配一个唯一的id，然后只允许按照升序规则来使用多个锁。

In [ ]:
import threading
from contextlib import contextmanager

# 存储已经请求锁的信息
_local = threading.local()


@contextmanager
def acquire(*locks):
    # 把锁通过id进行排序
    locks = sorted(locks, key=lambda x: id(x))
    acquired = getattr(_local, 'acquired', [])
    if acquired and max(id(lock) for lock in acquired) >= id(locks[0]):
        raise RuntimeError("Lock order Violation")
    acquired.extend(locks)
    _local.acquired = acquired

    try:
        for lock in locks:
            lock.acquire()
        yield
    finally:
        for lock in reversed(locks):
            lock.release()
        del acquired[-len(locks):]


x_lock = threading.Lock()
y_lock = threading.Lock()


def thread_1():
    for i in range(5):
        with acquire(x_lock,y_lock):
            print("Thread-1")


def thread_2():
    for i in range(5):
        with acquire(y_lock,x_lock):
            print("Thread-2")


t1 = threading.Thread(target=thread_1)
t1.daemon = True

t2 = threading.Thread(target=thread_2)
t2.daemon = True

t1.start()
t2.start()

t1.join()
t2.join()

通过排序，不管以什么样的顺序来请求锁，这些锁都会按照固定的顺序被获取。

这里用了thread.local()来保存请求锁的信息, 同样这个东西也可以用来保存线程的信息，而这个线程对其他的线程是不可见的